# Objetos - Prototipo y herencia

En esta etapa veremos lo que es el prototipo y herencia.

#### Antes revisaremos un particularidad entre `this` y las *arrow functions*

## Particularidad `this` y *arrow functions*

Durante la clase pasada revisamos que las clases podían crearse mediantes funciones, pero nos dimos cuenta que había una particularidad si utilizámamos *arrow functions*. Por ejemplo:

In [1]:
var personArrow = {
  name: "Jason",
  shout: () => console.log("My name is ", this.name)
}

personArrow.shout()

My name is  undefined


![alt text](http://derickbailey.com/wp-content/uploads/2015/09/undefined-this.jpeg "Title")

Source: https://derickbailey.com/2015/09/28/do-es6-arrow-functions-really-solve-this-in-javascript/

Wow! Pero ¿Qué pasó acá? ¿Pasará lo mismo si cambiamos de *arrow function* a función? Veamos:

In [2]:
var personFunction = {
  name: "Jason",
  shout: function() { console.log("My name is ", this.name) },
}

personFunction.shout()

My name is  Jason


(En este momento la cabeza explota)

Claramente aquí hay una diferencia y justamente tiene que ver con la referencia a `this`. En el caso de las funciones "normales" la referencia de `this` corresponde a la del contexto de la función (su propio `this`), mientras que en el caso de las *arrow functions* no hay una referencia al contexto propio, sino que más bien al entorno donde se ejecuta. Esto también se conoce como *Lexical Scope*.

**Algunos recursos:**

* https://medium.com/@jacobworrel/es6-arrow-functions-what-not-to-do-c28c96b4f396
* https://hackernoon.com/javascript-es6-arrow-functions-and-lexical-this-f2a3e2a5e8c4

# Lección: no usar arrow functions para definir métodos en el constructor de una clase

## Continuemos con objetos ahora :)

Al final de la clase anterior teníamos esto:

In [3]:
function Duck(name, age) {
    this.name = name;
    this.age = age;
    this.cuak = function(times) {
        while(times-- > 0) {
            console.log('Cuak!');
        }
    }
}

var duck1 = new Duck('Donald', 5);
var duck2 = new Duck('Lucas', 6);

Al realizar la siguiente comparación, notábamos algo muy particular:

In [4]:
duck1.cuak === duck2.cuak

false

¿Que quiere decir esto?¿Las dos funciones no son la misma?

![alt text](https://bikesforfish.files.wordpress.com/2015/09/patrick-surprised-face-tumblr-gallery-for-patrick-shocked-face-pictures.png?w=511 "Title")

Source: https://bikesforfish.com/2015/09/13/got-you-by-the-gubernaculum/

Lo anterior quiere decir que, cada vez que creemos un objeto `Duck`, estaremos creando también una nueva función ¿Ineficiente, no? ¿Cómo lo solucionamos? Aquí viene el **prototipo** a salvarnos!

### Lección: cuando sea un objeto duck, estamos creando un nuevo método dentro de el y no son iguales.

## Prototipo

Todos los objetos, **salvo el objeto base** tienen un prototipo.

**El prototipo es un objeto**, que tiene un prototipo!

## Las propiedades de un objeto se buscarán en el objeto, sino se encuentran ahí se buscarán en el prototipo, sino están ahí en el prototipo del prototipo y... puf! Herencia!

### ¿Cómo obtenemos el prototipo de un objeto?

Así:

In [5]:
// 3 formas de obtener el prototipo de un objeto:
// Forma1:
var prototypeOne = duck1.__proto__; 

// Forma2:
var prototypeTwo = duck1.constructor.prototype;// forma segura en todos los navegadores 
                                               // (incluso no modernos)

// Forma3:
var prototypeThree = Object.getPrototypeOf(duck1);

prototypeOne === prototypeTwo && prototypeTwo === prototypeThree;
// todas sirven para obtener el prototipo del duck1.

true

Para obtener el prototipo que una función le asigna a sus objetos:

In [6]:
console.log(Duck.prototype); // protipo la función (o clase ) Duck asgina a sus objetos.
Duck.prototype === prototypeTwo;

Duck {}


true

In [7]:
// Notamos que, es igual al de arriba
console.log(duck1.constructor.prototype);
console.log(duck2.constructor.prototype);
console.log(duck1.constructor.prototype === duck2.constructor.prototype);

console.log('--------------------');

// Pero el prototipo de su metodo correspondiente es distinto:
console.log(duck1.cuak.prototype);
console.log(duck2.cuak.prototype);
console.log(duck1.cuak.prototype === duck2.cuak.prototype);

Duck {}
Duck {}
true
--------------------
{}
{}
false


### ¿Cómo solucionamos el problema anterior?

Así:

In [8]:
function Duck(name, age) {
    this.name = name;
    this.age = age;
}

Duck.prototype.cuak = function(times) { // este va ser el prototipo 
    while(times-- > 0) {
        console.log('Cuak!');
    }
}

var duck1 = new Duck('Donald', 5);
var duck2 = new Duck('Lucas', 6);

[Function]

Y si probamos ahora:

In [9]:
duck1.cuak === duck2.cuak

true

Ahora si!

In [10]:
// Viendo algunas cosas:
console.log(Duck.prototype); // ahora el prototipo que asigna a sus objetos tienen ambos el mismo método

// siguen siendo iguales
console.log(duck1.constructor.prototype);
console.log(duck2.constructor.prototype);

console.log(duck1.constructor.prototype === duck2.constructor.prototype)

Duck { cuak: [Function] }
Duck { cuak: [Function] }
Duck { cuak: [Function] }
true


In [11]:
// Pero AHORA EL PROTOTIPO DE SU MÉTODO ES IGUAL:
console.log(duck1.cuak.prototype);
console.log(duck2.cuak.prototype);
console.log(duck1.cuak.prototype === duck2.cuak.prototype);

{}
{}
true


## Herencia

Bueno ¿Y cómo hacemos herencia?

Vamos a crear mamíferos que se pueden mover de distinta forma! Observa el siguiente ejemplo:

In [12]:
// Creamos la función Mamífero
function Mammal(name) {
    this.name = name;
}

// Agregamos a su prototipo la funcion moverse
// En otras palabras: Agrego un método moverse y que va ser igual en todos los objetos
// de clase mamífero.
Mammal.prototype.move = function() {
    console.log("Mammal is moving...");
}

//----------------------------------------------------------------

// Creamos una funcion Perro
function Dog(name, breed) {
    Mammal.call(this, name); // Call(<objeto>, <p1>, ..., <pn> )
    this.breed = breed;
}
//----------------------------------------------------------------


// Creamos una funcion Delfin
function Dolphin(name) {
    Mammal.call(this, name); // Call(<objeto>, <p1>, ..., <pn> )
}

//----------------------------------------------------------------

// Asignamos a su prototipo un Mamífero
Dog.prototype = new Mammal();
Dolphin.prototype = new Mammal();
//----------------------------------------------------------------


var dog = new Dog("Snoopy", "Beagle");
var dolphin = new Dolphin("Willy");

dog.move();
dolphin.move();

// Alteramos el prototipo del delfín
Dolphin.prototype.move = function() {
    console.log("Dolphin is swimming...");
}

dog.move();
dolphin.move();

Mammal is moving...
Mammal is moving...
Mammal is moving...
Dolphin is swimming...


## Lo anterior: es herencia pero de forma enredada

Y, tal como vimos la clase pasada, al ser incorporado `class` el realizar clases es mucho más simple!

In [13]:
class Cat extends Mammal{
    constructor(name){
        super(name);
    }
}

cat = new Cat();
cat.move();

Mammal is moving...


## Ejercicios

### Ejercicio 1

Represente la siguiente situación utilizando lo aprendido anteriormente:

Tu tienes distintos tipos de vehículos: auto, moto y bicicleta. Representa esto con clases considerando lo siguiente:

* Todos los vehículos pueden moverse
* Sólo el auto y la moto pueden encenderse
* El auto y la moto tiene nivel de estanque (número de 1 a 100)
* La bicicleta tiene una luz trasera, que puede encenderse o apagarse

Para todas las acciones anteriores basta con que muestre un mensaje en consola para indicar su estado.

In [36]:
// Tu solución aquí
class Vehiculo2 {
    
    mover(){
        console.log("me estoy moviendo")
    }
}


class Auto2 extends Vehiculo2 {
    constructor(){
        super()
        this.status = false; // false (apagado), true (encendido)
        this.nivel_de_estanque = 1;
    }
    
    encender(){
        this.status = true;
    }
}

class Moto2 extends Auto2 {
    constructor(){
        super()
    }
}


class Bicicleta2 extends Vehiculo2 {
    constructor(){
        super()
        this.luz_trasera = 'apagada'; // false (apagado), true (encendido)
    }
    
    encender_luz_trasera(){
        console.log('Se ha prendido al luz trasera de la bicicleta');
        this.luz_trasera = 'prendida'
    }
}

In [43]:
var ve1 = new Vehiculo2();

var a1 = new Auto2();

var m1 = new Moto2();

var b1 = new Bicicleta2();


// 1: Todos los vehículos pueden moverse:
ve1.mover();
a1.mover();
m1.mover();
b1.mover();
console.log('--------------')
// 2: Sólo el auto y la moto pueden encenderse
console.log('Auto: ')
console.log(a1.status);
a1.encender();
console.log(a1.status);

console.log('Moto: ')
console.log(m1.status);
m1.encender();
console.log(m1.status);

// ve1.encender() // error: ve1.encender is not a function
// b1.encender(); // error: b1.encender is not a function

console.log('------')
// 3: El auto y la moto tiene nivel de estanque (número de 1 a 100)

console.log(ve1.nivel_de_estanque); // no tiene (undefined)
console.log(a1.nivel_de_estanque);  
console.log(b1.nivel_de_estanque); // no tiene (undefined)
console.log(m1.nivel_de_estanque);

console.log('-------')

// 4: La bicicleta tiene una luz trasera, que puede encenderse o apagarse

console.log(b1.luz_trasera);
b1.encender_luz_trasera();
console.log(b1.luz_trasera);

console.log(ve1.luz_trasera); // no tiene luz trasera
console.log(a1.luz_trasera);  // no tiene 
console.log(m1.luz_trasera); // no tiene


me estoy moviendo
me estoy moviendo
me estoy moviendo
me estoy moviendo
--------------
Auto: 
false
true
Moto: 
false
true
------
undefined
1
undefined
1
-------
apagada
Se ha prendido al luz trasera de la bicicleta
prendida
undefined
undefined
undefined


### Solución profe:

In [55]:
// Tu solución aquí

// Vehicle
function Vehicle(type) {
    this.type = type;
}

Vehicle.prototype.move = function() {
    console.log(`Vehicle ${this.type} is moving`);
}

// Car and Motorcycle
function Car(fuel) {
    Vehicle.call(this, 'car');
    this.fuel = fuel;
}

function Motorcycle(fuel) {
    Vehicle.call(this, 'motorcycle');
    this.fuel = fuel;
}

function turnOn() {
    console.log('Stating engine');
}

Car.prototype = new Vehicle();
Motorcycle.prototype =  new Vehicle();

Car.prototype.turnOn = turnOn;
Motorcycle.prototype.turnOn = turnOn;

// Bike

function Bike(){
    Vehicle.call(this, 'bike');
    this.isBackLightOn = false;
}

Bike.prototype = new Vehicle();
Bike.prototype.turnLightOn = function() {
    this.isBackLightOn = true;
}
Bike.prototype.turnLightOff = function() {
    this.isBackLightOn = false;
}

var bike = new Bike();
bike.move();
console.log(bike.fuel); // no tiene, y sale undefined. entonces esta bien 
// lo que hice arriba.

Vehicle bike is moving
undefined


### Ejercicio 2
# REVISAR PENDIENTE entender mejor

¿Podrías intentar determinar el nivel de acceso de esta clase? Existen privados, privilegiados y públicos:

```javascript
// Para los parámetros manufacturer y model
function Smartphone(manufacturer, model) {
  this.manufacturer = manufacturer;
  this.model = model;
  // Para la función getModel
  this.getModel = function() {
    return this.model;
  }
}

// Para la función ring
Smartphone.prototype.ring = function() {
  console.log('Ring...ring...ring');
}

```

In [44]:
// Para los parámetros manufacturer y model
function Smartphone(manufacturer, model) {
  this.manufacturer = manufacturer;
  this.model = model;
  // Para la función getModel
  this.getModel = function() {
    return this.model;
  }
}

// Para la función ring
// agregar el metodo ring, para todos los objetos de clase Smartphone:
Smartphone.prototype.ring = function() {
  console.log('Ring...ring...ring');
}

[Function]

In [56]:
// Solución del profe:
// 1. manufacturer: es privado, solo se pueden acceder los
// que tengan su scope/closure

// 2. model: es privilegiado, es de acceso publico ( con getmodel() ), pero
// puede acceder a propiedades privadas.


// 3. las funciones asignadas al prototipo, (fuera del constructor) 
// son de tipo publicas. No podran accer a propiedades privadas,
// pero si a las que son privilegiadas.


In [57]:
// En la practica.. wtf? no caché
var s1 = new Smartphone('Nokia', '1234');
console.log(s1.manufacturer); // es publico.

console.log(s1.model); // es publico
console.log(s1.getModel()); // ? tener un getter? que lo hace, si igual se puede acceder publicamente

s1.ring(); // metodo publico, atambipen puedo acceder altiro

Nokia
1234
1234
Ring...ring...ring


# Al parecer agregando '_' antes de los atributos, tiene sentido todo:

In [58]:
// Para los parámetros manufacturer y model
function Smartphone2(manufacturer, model) {
  this._manufacturer = manufacturer;
  this._model = model;
  // Para la función getModel
  this.getModel = function() {
    return this._model;
  }
}

// Para la función ring
// agregar el metodo ring, para todos los objetos de clase Smartphone:
Smartphone2.prototype.ring = function() {
  console.log('Ring...ring...ring');
}

[Function]

In [64]:
// En la practica.. wtf? no caché
var s1 = new Smartphone2('Nokia', '1234');
var s2 = new Smartphone2('Huawei', '9876');
console.log(s1.manufacturer);
// es privado: lo puedo llamar solo dentro de la clase, con 'this'
console.log(s1.model); 
// privilegiado, atributo que lo puedo llamar solo dentro de la clase con 'this'
// o por otro objeto de la misma clase (pero siempre dentro de su constructor)
console.log(s1.getModel()); // hacer un getter lo hace privilegiado.

s1.ring(); // metodo publico, lo puedo llamar dentro y fuera de la clase.

undefined
undefined
1234
Ring...ring...ring


In [ ]:
// Solución profe: 
// Para el primer caso, son de tipo privado. 
// Sólo podrán ser accesibles para aquellos que los tengan en su scope/closure.

// Para el segundo caso, la función tiene un nivel de acceso provilegiado.
// Serán de acceso público, pero podrán acceder a las propiedades privadas.

// Finalmente, las funciones que se asignen al prototipo (fuera del constructor)
// serán de tipo públicas.
// No podrán acceder a las propiedades privadas pero sí a aquellas 
// de tipo provilegiadas